In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from case_hours import *
from pract2_utils import *

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from collections import Counter

# Case 2

Accidentes que se producen a la noche y de madrugada en una proviciona concreta, por ejemplo Madrid, que tiene la mayor densidad de población de España.

In [ ]:
sns.set_style("whitegrid")

In [ ]:
case_2_subset = define_case_2()

In [ ]:
case_2_subset.head()

In [ ]:
case_2_subset.shape

In [ ]:
case_2_subset['provincia'].value_counts().head()

In [ ]:
case_2_subset['diasemana'].value_counts().head()

In [ ]:
case_2_subset['hora'].value_counts().head()

In [ ]:
atributos = ['tot_vehiculos_implicados', 'tot_heridos_leves', 'tot_heridos_graves','tot_muertos', 'tot_victimas']

In [ ]:
hm = sns.pairplot(case_2_subset, vars = atributos, diag_kind="kde")
hm.savefig("figures/2_segmentation/case_2/eda.png")

In [ ]:
case_2_data = to_matrix(case_2_subset, atributos)

In [ ]:
case_2_data_norm = norm(case_2_data)

In [ ]:
# Definicion of algorithms for study
case_2_algorithms = definition_clusters(case_2_data_norm)

In [ ]:
case_2_predictions, case_2_times = get_predictions(case_2_algorithms, case_2_data_norm)

In [ ]:
case_2_measures = calcule_measures(case_2_data_norm, case_2_predictions, case_2_times)

In [ ]:
case_2_measures

In [ ]:
columns = ['Algoritmo', 'Clusters', 'Calinski', 'Silhouette', 'time(s)']
print(latex_table(case_2_measures, columns, False))

In [ ]:
a, b, n = 1, 10, 1

In [ ]:
# Elbow
case_2_distortions, case_2_inertia, case_2_time_kmeans  = kmeans_parameters(case_2_data_norm, a, b, n)
plot_time(case_2_inertia, case_2_time_kmeans, "elbow", "time(s)", "figures/2_segmentation/case_2/kmeans_elbow.pdf", a, b, n)

In [ ]:
a, b, n = 5, 50, 5

In [ ]:
# Distortions
case_2_distortions, case_2_inertia, case_2_time_kmeans  = kmeans_parameters(case_2_data_norm, a, b, n)

In [ ]:
plot_time(case_2_distortions, case_2_time_kmeans, "Distortions", "time(s)", "figures/2_segmentation/case_2/kmeans_distortions.pdf", a, b, n)

In [ ]:
# Inertia
plot_time(case_2_inertia, case_2_time_kmeans, "Inertia", "time(s)", "figures/2_segmentation/case_2/kmeans_inertia.pdf", a, b, n)

In [ ]:
case_2_silhouette_scores, case_2_calinski_scores = measures_kmeans_range(case_2_data_norm, a, b, n)

In [ ]:
# Comparison silhouette calinski
plot_time(case_2_silhouette_scores, case_2_calinski_scores, "silhouette", "calinski", "figures/2_segmentation/case_2/silhouette_calinski.pdf", a, b, n)

In [ ]:
case_2_table = {
                "Silhouette":case_2_silhouette_scores,
                "Calinski-Harabaz":case_2_calinski_scores,
                "time(s)":case_2_time_kmeans
                }

In [ ]:
columns = ['Silhouette','Calinski-Harabaz','time(s)']
index=range(a,b,n)
print(latex_table_index(case_2_table, columns, index))

## Outliers

In [ ]:
# Generamos el nuevo set sin outliers
ward_no_outliers_subset = delete_outliers(case_2_predictions[3], case_2_subset, 2)

In [ ]:
ward_no_outliers_subset_data_norm = norm(to_matrix(ward_no_outliers_subset, atributos))

In [ ]:
ward_no_outliers_subset_prediction, ward_no_outliers_times = get_predictions(case_2_algorithms[3:5], ward_no_outliers_subset_data_norm)

In [ ]:
ward_no_outliers_measures = calcule_measures(ward_no_outliers_subset_data_norm, ward_no_outliers_subset_prediction, ward_no_outliers_times)

In [ ]:
# 35
ward_35 = AgglomerativeClustering(n_clusters=35, linkage='ward')
prediction_ward_35 =  ('ward_35', ward_35.fit_predict(ward_no_outliers_subset_data_norm))

In [ ]:
ward_35_no_outliers_subset = delete_outliers(prediction_ward_35, ward_no_outliers_subset, 2)

In [ ]:
case_2_agglomerative = configuraciones_agglomerative()

In [ ]:
ward_35_no_outliers_data_norm = norm(to_matrix(ward_35_no_outliers_subset, atributos))

In [ ]:
case_ward_35_predictions_agglomerative, case_ward_35_times_agglomerative  = get_predictions(case_2_agglomerative, ward_35_no_outliers_data_norm)

In [ ]:
case_35_measures_ward = calcule_measures(ward_35_no_outliers_data_norm, case_ward_35_predictions_agglomerative, case_ward_35_times_agglomerative)

In [ ]:
# Tabla de medidas sin outliers
columns = ['Algoritmo', 'Clusters', 'Calinski', 'Silhouette', 'time(s)']
print(latex_table(case_35_measures_ward, columns, False))

## Interpretación

In [ ]:
# kmeans
kmeans_label = case_2_predictions[0][1]

In [ ]:
Counter(kmeans_label)

In [ ]:
pairplot(case_2_subset, atributos, "figures/2_segmentation/case_2/pairplot_kmeans.png", kmeans_label)

In [ ]:
df_kmeans_centroids = pd.DataFrame(case_2_data_norm)
df_kmeans_centroids.columns = atributos
df_kmeans_centroids['cluster'] = kmeans_label
df_kmeans_centroids = df_kmeans_centroids.groupby('cluster').mean()

In [ ]:
visualize_centroids(df_kmeans_centroids.values, case_2_data_norm, "figures/2_segmentation/case_2/centroids_kmeans_norm.pdf", atributos, 0.0)

In [ ]:
visualize_centroids(df_kmeans_centroids.values, case_2_data, "figures/2_segmentation/case_2/centroids_kmeans.pdf", atributos, 0.0)

In [ ]:
ward_35_labels = case_ward_35_predictions_agglomerative[0][1]

In [ ]:
pairplot(ward_35_no_outliers_subset, atributos, "figures/2_segmentation/case_2/pairplot_ward35.png", ward_35_labels)

In [ ]:
# 5
ward_5 = AgglomerativeClustering(n_clusters=5, linkage='ward')
prediction_ward_5 =  ('ward_5', ward_5.fit_predict(ward_no_outliers_subset_data_norm))

In [ ]:
ward_5_no_outliers_subset = delete_outliers(prediction_ward_5, ward_no_outliers_subset, 2)

In [ ]:
ward_5_no_outliers_data = to_matrix(ward_5_no_outliers_subset, atributos)

In [ ]:
ward_5_no_outliers_data_norm = norm(ward_5_no_outliers_data)

In [ ]:
case_ward_5_predictions_agglomerative, case_ward_5_times_agglomerative  = get_predictions(case_2_agglomerative, ward_5_no_outliers_data_norm)

In [ ]:
case_5_measures_ward = calcule_measures(ward_5_no_outliers_data_norm, case_ward_5_predictions_agglomerative, case_ward_5_times_agglomerative)

In [ ]:
ward_5_labels = case_ward_5_predictions_agglomerative[0][1]

In [ ]:
pairplot(ward_5_no_outliers_subset, atributos, "figures/2_segmentation/case_2/pairplot_ward5.png", ward_5_labels)

In [ ]:
# setting distance_threshold=0 ensures we compute the full tree.
model = AgglomerativeClustering(linkage='ward', distance_threshold=0, n_clusters=None)

model = model.fit(ward_35_no_outliers_data_norm)
plt.title('Hierarchical Clustering Dendrogram')
# plot the top three levels of the dendrogram
plot_dendrogram(model, truncate_mode='level', p=4)
#plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.savefig("figures/2_segmentation/case_2/hierarchical_clustering.pdf")
plt.show()

In [ ]:
dendograma_subset = pd.DataFrame(ward_35_no_outliers_subset,index=ward_35_no_outliers_subset.index, columns=atributos)
hm = sns.clustermap(dendograma_subset, method='ward', col_cluster=False, figsize=(20,10), cmap="YlGnBu",  yticklabels=False)
hm.savefig("figures/2_segmentation/case_2/clustermap.pdf")

In [ ]:
Counter(ward_35_labels)

In [ ]:
Counter(ward_5_labels)

In [ ]:
df_ward_5_no_outliers = pd.DataFrame(ward_5_no_outliers_data_norm)
df_ward_5_no_outliers.columns = atributos
df_ward_5_no_outliers['cluster'] = ward_5_labels
df_ward_5_centroides = df_ward_5_no_outliers.groupby('cluster').mean()

In [ ]:
visualize_centroids(df_ward_5_centroides.values, ward_5_no_outliers_data_norm, "figures/2_segmentation/case_2/centroids_ward_5_norm.pdf", atributos, 0.0)

In [ ]:
visualize_centroids(df_ward_5_centroides.values, ward_5_no_outliers_data, "figures/2_segmentation/case_2/centroids_ward_5.pdf", atributos, 0.0)